In [1]:
import pandas as pd
import numpy as np
import sys
from common import commons
home = commons.home
from features_preprocess import get_winid
import os
import re

In [2]:
def nearest_tss(tss,sites_df):
    merged = pd.merge(sites_df,tss,how='outer',on=['chr','coordinate'])
    merged.sort_values(['chr','coordinate'],inplace=True)
    merged.rename(columns={'strand':'before_tss'},inplace=True)
    merged.ix[merged['before_tss'].isnull()==False, 'before_tss'] = merged.ix[merged['before_tss'].isnull()==False,'coordinate']
    merged['after_tss'] = merged['before_tss']
    merged['before_tss'].fillna(method='ffill', inplace=True)
    merged['after_tss'].fillna(method='bfill',inplace=True)
    merged['dist_to_before_tss'] = np.abs(merged['coordinate']-merged['before_tss'])
    merged['dist_to_after_tss'] = np.abs(merged['coordinate']-merged['after_tss'])
    merged['tss'] = None
    before_ix = (merged['dist_to_before_tss'] < merged['dist_to_after_tss']) | (merged['dist_to_after_tss'].isnull())
    merged.ix[before_ix,'tss'] = merged.ix[before_ix,'before_tss']
    after_ix = (merged['dist_to_before_tss'] >= merged['dist_to_after_tss']) | (merged['dist_to_before_tss'].isnull())
    merged.ix[after_ix,'tss'] = merged.ix[after_ix,'after_tss']
    merged['dist_to_nearest_tss'] = np.abs(merged['coordinate']-merged['tss']) 
    merged.drop(['before_tss','after_tss','tss','dist_to_before_tss','dist_to_after_tss'],axis=1,inplace=True)
    merged.dropna(axis=0,subset=['id'],inplace=True)
    return merged

def rename_features(x):   #rename repetitive features
    features = np.array(x.columns)
    features_count = pd.Series(index=x.columns.unique())
    features_count = features_count.fillna(int(0))
    for i,name in enumerate(x.columns):
        if features_count[name] == 0:
            features_count[name] += 1
        else:
            features[i] = name+str(features_count[name])
            features_count[name] += 1
    x.columns = features
    return 

In [3]:
dataset = 'AD_CpG'
if dataset == 'AD_CpG':
    type_name = commons.type_name  ## amyloid, cerad, tangles
    with_cell_type = commons.with_cell_type ## with or without
    dataset = dataset+'/'+type_name+with_cell_type
with pd.HDFStore(home+'data/'+dataset+'/all_sites_winid','r') as h5s:
    all_sites = h5s['all_sites_winid']
all_sites.reset_index(drop=True,inplace=True)

In [4]:
all_sites

,id,chr,coordinate,beta_sign,pvalue,beta,label,start,winid,end
0,cg14008593,1,1117473.0,-0.619262,0.536040,0.736632,0,1117401,5588,1117600
1,cg19827148,1,1260496.0,-0.015111,0.987950,0.015523,0,1260401,6303,1260600
2,cg06817090,1,1334334.0,-0.651393,0.515107,0.042461,0,1334201,6672,1334400
3,cg01189072,1,1610660.0,-0.651154,0.515261,0.943234,0,1610601,8054,1610800
4,cg11961999,1,2104478.0,0.223292,0.823404,0.714269,0,2104401,10523,2104600
5,cg23441090,1,2126425.0,-0.631741,0.527860,0.009729,0,2126401,10633,2126600
6,cg18771653,1,2126479.0,-0.822940,0.410954,0.020238,0,2126401,10633,2126600
7,cg07333862,1,2158915.0,-0.424742,0.671217,0.077125,0,2158801,10795,2159000
8,cg04619852,1,2234433.0,0.343825,0.731130,0.540769,0,2234401,11173,2234600
9,cg08655953,1,2425888.0,-0.797143,0.425765,0.968990,0,2425801,12130,2426000


In [5]:
feature_dir = home+'data/features/'+dataset+'/'
files = os.listdir(feature_dir)
pattern = '.*all.csv$'
reg = re.compile(pattern)
files = [name for name in files if len(reg.findall(name))>0]




In [6]:
for file in files:    
    feature = pd.read_csv(feature_dir+file)
    print(len(feature.columns))
    all_sites = pd.concat([all_sites,feature],axis=1)

31
267
317
73
80
735
303


In [7]:
rename_features(all_sites)

In [8]:
all_sites.shape

(1529, 1816)

In [9]:
columns = all_sites.columns.values

In [10]:
columns = all_sites.columns.values
pattern = re.compile(r'.*WGBS.*')
i=0
for col in columns:
    if len(pattern.findall(col))>0:
        print(col)
        i += 1
print(i)

0


In [11]:
all_sites.drop(['start','end'],axis=1,inplace=True)

In [12]:
additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS']
#merge with additional features
with pd.HDFStore(feature_dir+'addtional_features','r') as h5s:
    for feature in additional_features:
        feature_frame = h5s[feature]
        all_sites = pd.concat([all_sites,feature_frame],axis=1)
all_sites = all_sites.loc[:,~all_sites.columns.duplicated()]
all_sites['chr'] = all_sites['chr'].astype('i8')

In [13]:
all_sites.ix[:,:1950]

/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


,id,chr,coordinate,beta_sign,pvalue,beta,label,winid,A549,Astrocy,...,ENCFF723ZMR_RNASeq_counts,ENCFF301ROZ_RNASeq_counts,ENCFF888ZFS_RNASeq_counts,ENCFF105THO_RNASeq_counts,ENCFF760IDU_RNASeq_counts,ENCFF624VBI_RNASeq_counts,ENCFF552FTX_RNASeq_counts,ENCFF623UTC_RNASeq_counts,ENCFF535JQR_RNASeq_counts,ENCFF003JVR_WGBS_counts
0,cg14008593,1,1117473.0,-0.619262,0.536040,0.736632,0,5588,3,2,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0
1,cg19827148,1,1260496.0,-0.015111,0.987950,0.015523,0,6303,3,4,...,38.0,7.0,7.0,89.0,3.0,24.0,13.0,11.0,21.0,0.0
2,cg06817090,1,1334334.0,-0.651393,0.515107,0.042461,0,6672,2,2,...,1.0,2.0,3.0,23.0,6.0,34.0,12.0,31.0,39.0,0.0
3,cg01189072,1,1610660.0,-0.651154,0.515261,0.943234,0,8054,9,9,...,0.0,0.0,0.0,0.0,3.0,11.0,1.0,1.0,7.0,10.0
4,cg11961999,1,2104478.0,0.223292,0.823404,0.714269,0,10523,2,5,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,2.0,2.0,11.0
5,cg23441090,1,2126425.0,-0.631741,0.527860,0.009729,0,10633,5,7,...,2.0,0.0,0.0,12.0,2.0,0.0,1.0,0.0,4.0,0.0
6,cg18771653,1,2126479.0,-0.822940,0.410954,0.020238,0,10633,5,7,...,2.0,0.0,0.0,12.0,2.0,0.0,1.0,0.0,4.0,0.0
7,cg07333862,1,2158915.0,-0.424742,0.671217,0.077125,0,10795,1,5,...,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,2.0,0.0
8,cg04619852,1,2234433.0,0.343825,0.731130,0.540769,0,11173,5,3,...,756.0,20.0,0.0,251.0,10.0,162.0,36.0,141.0,54.0,5.0
9,cg08655953,1,2425888.0,-0.797143,0.425765,0.968990,0,12130,9,6,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,4.0,1.0,0.0


In [14]:
#nearest tss distance    
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
all_sites = nearest_tss(tss,all_sites)


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [15]:
with pd.HDFStore(home+'data/'+dataset+'/all_features','w') as h5s:
    h5s['all_features'] = all_sites

In [16]:
###all 450K sites features
dataset = 'AD_CpG'
if dataset == 'AD_CpG':
    type_name = commons.type_name  ## amyloid, cerad, tangles
    with_cell_type = commons.with_cell_type ## with or without
    dataset = dataset+'/'+type_name+with_cell_type


In [18]:
with pd.HDFStore(home+'data/'+dataset.split('/')[0]+'/all_450k_sites_winid','r') as h5s:
    all_sites = h5s['all_450k_sites_winid']
all_sites.reset_index(drop=True,inplace=True)

In [19]:
all_sites

,id,chr,coordinate,pvalue,beta,start,winid,end
0,cg13869341,1,15865.0,0.977843,0.877442,15801,80,16000
1,cg24669183,1,534242.0,0.818908,0.786274,534201,2672,534400
2,cg15560884,1,710097.0,0.843290,0.652625,710001,3551,710200
3,cg01014490,1,714177.0,0.445636,0.008987,714001,3571,714200
4,cg17505339,1,720865.0,0.211757,0.859844,720801,3605,721000
5,cg11954957,1,758829.0,0.139721,0.816452,758801,3795,759000
6,cg23803172,1,763119.0,0.667176,0.004858,763001,3816,763200
7,cg16736630,1,779995.0,0.338102,0.758187,779801,3900,780000
8,cg05898754,1,805102.0,0.502358,0.359355,805001,4026,805200
9,cg03128332,1,805338.0,0.439849,0.142779,805201,4027,805400


In [20]:
feature_dir = home+'data/features/'+dataset+'/'
files = os.listdir(feature_dir)
pattern = '.*all_450k.csv$'
reg = re.compile(pattern)
files = [name for name in files if len(reg.findall(name))>0]

In [21]:
for file in files:    
    feature = pd.read_csv(feature_dir+file)
    print(len(feature.columns))
    all_sites = pd.concat([all_sites,feature],axis=1)

317
267
80
31
735
303
73


In [22]:
rename_features(all_sites)

In [23]:
all_sites.drop(['start','end'],axis=1,inplace=True)

In [24]:
additional_features = ['ATAC','CADD','DANN','Eigen','GenoCanyon','RNASeq','WGBS']
#merge with additional features
with pd.HDFStore(feature_dir+'all_450k_addtional_features','r') as h5s:
    for feature in additional_features:
        feature_frame = h5s[feature]
        all_sites = pd.concat([all_sites,feature_frame],axis=1)
all_sites = all_sites.loc[:,~all_sites.columns.duplicated()]
all_sites['chr'] = all_sites['chr'].astype('i8')

In [25]:
#nearest tss distance 
chrs = all_sites['chr'].unique()
cols=['chr', 'coordinate','strand']
tss =  pd.read_csv(home+'tss.txt',sep='\s+',header=None,names=cols,skiprows=1)
tss = get_winid.convert_chr_to_num(tss,chrs)
tss.sort_values(['chr','coordinate'],inplace=True)
all_sites = nearest_tss(tss,all_sites)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']


/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [26]:
all_sites

,id,chr,coordinate,pvalue,beta,winid,A549-ATF3,A549-BCL3,A549-CEBPB,A549-CREB1,...,NCFF795DNO_WGBS_counts,NCFF801OHX_WGBS_counts,NCFF811QOG_WGBS_counts,NCFF831OYO_WGBS_counts,NCFF843SYR_WGBS_counts,NCFF847OWL_WGBS_counts,NCFF874GGB_WGBS_counts,NCFF913ZNZ_WGBS_counts,NCFF923CZC_WGBS_counts,dist_to_nearest_tss
0,cg13869341,1,15865.0,0.977843,0.877442,80.0,4.0,11.0,0.0,0.0,...,0.0,13.0,0.0,7.0,0.0,0.0,11.0,3.0,0.0,3991.0
1,cg24669183,1,534242.0,0.818908,0.786274,2672.0,6.0,11.0,1.0,1.0,...,0.0,16.0,14.0,4.0,9.0,2.0,9.0,12.0,12.0,87792.0
2,cg15560884,1,710097.0,0.843290,0.652625,3551.0,3.0,3.0,0.0,2.0,...,3.0,72.0,74.0,32.0,83.0,38.0,18.0,62.0,21.0,3971.0
3,cg01014490,1,714177.0,0.445636,0.008987,3571.0,24.0,24.0,4.0,2371.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,109.0
4,cg17505339,1,720865.0,0.211757,0.859844,3605.0,2.0,0.0,0.0,0.0,...,7.0,5.0,28.0,17.0,35.0,23.0,30.0,21.0,16.0,6797.0
5,cg11954957,1,758829.0,0.139721,0.816452,3795.0,3.0,2.0,0.0,0.0,...,3.0,54.0,26.0,33.0,19.0,31.0,10.0,13.0,7.0,4073.0
6,cg23803172,1,763119.0,0.667176,0.004858,3816.0,13.0,10.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0
7,cg16736630,1,779995.0,0.338102,0.758187,3900.0,2.0,1.0,0.0,2.0,...,2.0,71.0,23.0,13.0,16.0,26.0,32.0,15.0,9.0,16817.0
8,cg05898754,1,805102.0,0.502358,0.359355,4026.0,6.0,5.0,5.0,10.0,...,1.0,7.0,5.0,3.0,3.0,0.0,8.0,2.0,2.0,7080.0
9,cg03128332,1,805338.0,0.439849,0.142779,4027.0,13.0,10.0,2.0,51.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6844.0


In [27]:
with pd.HDFStore(home+'data/AD_CpG/all_450k_features','w') as h5s:
    h5s['all_450k_features'] = all_sites